# Name: Ayesha Zafar
## Registration No. : (MSDS23011)
## Git Repository URL: https://github.com/Ayesha-Zafar1/Gradio_Rag_Chatbot
## Gradio App on Huggingface URL: https://huggingface.co/spaces/Ayesha-15/gradio-rag-chatbot

In [2]:
!pip install unstructured
!pip install pdfminer.six
!pip install pillow-heif
!pip install pi-heif
!pip install unstructured-inference
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install unstructured-pytesseract
!apt-get install -y tesseract-ocr
!tesseract --version
!pip install unstructured-pytesseract



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.1 MB/s eta 0:00:00
  

In [3]:
!pip install sentence-transformers
!pip install pinecone-client
!pip install groq
!pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00


In [4]:
!pip install --upgrade unstructured


In [11]:
!pip install ragas

In [12]:
from unstructured.partition.pdf import partition_pdf
#from unstructured.documents.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
#from gradio import gr
from IPython.display import display
import requests

In [13]:
import os
from pdf2image import convert_from_path
from pdfminer.high_level import extract_text
from PIL import Image
import pytesseract
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from google.colab import files
import json

## Task 1: Process the PDF

In [ ]:

# # Partition PDF and save structured data
# pdf_path = "NetSol_Financial Statement_2024_Part 1.pdf"
# elements = partition_pdf(filename=pdf_path)
# with open("structured_data.json", "w") as f:
#     json.dump([el.to_dict() for el in elements], f, indent=4)


In [15]:
# Upload PDF
uploaded = files.upload()
pdf_path = next(iter(uploaded.keys()))


Saving NetSol_Financial Statement_2024_Part 1.pdf to NetSol_Financial Statement_2024_Part 1 (2).pdf


PROJECT

In [16]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        text = extract_text(pdf_path)
        if text.strip():
            return text
    except Exception as e:
        print("Failed to extract text using pdfminer:", e)

    # Fallback to OCR if pdfminer fails
    print("Using Tesseract for OCR on scanned PDF...")
    images = convert_from_path(pdf_path)
    text = ""
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ+-*/=()[]{}^~<>πσ√'

    for i, image in enumerate(images):
        text += pytesseract.image_to_string(image, config=custom_config) + "\n"

    return text

# # Upload PDF
# uploaded = files.upload()  # Prompt for file upload
# pdf_path = next(iter(uploaded.keys()))

# Extract text from the uploaded PDF
extracted_text = extract_text_from_pdf(pdf_path)

# Split the extracted text into chunks using LangChain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.create_documents([extracted_text])

# Load Sentence-BERT model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode([text.page_content for text in texts])

# Generate embeddings for the chunks
embeddings = generate_embeddings(chunks)

# Print the embeddings
print("Generated Embeddings:")
for i, embedding in enumerate(embeddings):
    print(f"Chunk {i + 1}: {embedding[:10]}...")  # Print first 10 values of each embedding for brevity

Generated Embeddings:
Chunk 1: [ 0.01572167 -0.04187086 -0.01129073  0.01721135  0.01138737  0.02450416
  0.0302611  -0.04534864 -0.00633959  0.00877455]...
Chunk 2: [-0.0078191   0.04579374 -0.00520206  0.05025609 -0.03695016  0.04508304
  0.09698196 -0.05948566  0.02933256 -0.02643448]...
Chunk 3: [-0.00745231 -0.01312333 -0.05819414 -0.07115336 -0.10053743  0.0199601
  0.0974448  -0.03394496 -0.00176364 -0.02687869]...
Chunk 4: [ 0.02686614  0.00238659 -0.04720411 -0.07657102  0.00390575  0.04014663
  0.08422335 -0.00949266  0.03953831 -0.03970661]...
Chunk 5: [-0.00487514  0.05378122 -0.06915488  0.00047935 -0.0076402   0.02700915
  0.07109533 -0.00899707 -0.04658901 -0.1002297 ]...
Chunk 6: [ 0.00448836  0.02885237 -0.03607311 -0.0035545  -0.03732537  0.02050648
  0.12028529 -0.04650278  0.0096084  -0.04166941]...
Chunk 7: [-0.0063596   0.09786002 -0.0715922  -0.07467435 -0.03813307  0.06834555
  0.10766438 -0.03698289  0.00116889 -0.06149121]...
Chunk 8: [ 0.00503337  0.03333671 

In [17]:
# Get the dimension of the embeddings
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

Embedding Dimension: 384


## Task 2: Develop Query Routing

In [11]:
def classify_query(user_query: str) -> str:
    # Simple classification based on keywords
    if any(keyword in user_query.lower() for keyword in ["finance", "stock", "market", "investment", "financial", "NetSol"]):
        return "finance"
    elif any(keyword in user_query.lower() for keyword in ["news", "current event", "live event", "happening now"]):
        return "live-event"
    else:
        return "general"

# Example
user_query = input("Please enter your query: ")
query_category = classify_query(user_query)
print(f"Classified as: {query_category}")


Please enter your query: Explain the key financial metrics for NetSol in 2024.
Classified as: finance


## Task 3: Build RAG Workflow

Set Up Pinecone

In [18]:
from pinecone import Pinecone, ServerlessSpec, Index
from sentence_transformers import SentenceTransformer
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_8drpB_AMNSB8Tnur3jrKkt9egkSfezH8xGuDcToYaCQ5e9qarYAmXqor2w6uY5ph9qV1s")  # Use your actual API key

# List indexes
indexes = pc.list_indexes().names()
print("Indexes:", indexes)

# If an index exists, describe it, else create a new index
if indexes:
    index_name = indexes[0]  # Access the first index in the list and get the 'name'
    index_description = pc.describe_index(index_name)  # Pass the index name as a string
    print(f"Details of index '{index_name}':", index_description)
else:
    # Create a new index if none exist
    pc.create_index(
        name='netsol_financials',
        dimension=384,  # Update dimension according to your embedding model
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Set your region here
        )
    )
    print("Created 'netsol_financials' index.")

# Access the 'notes' index using the Index class
index = Index(name='netsol_financials', host='https://netsol-financials-c2umrz9.svc.aped-4627-b74a.pinecone.io', api_key="pcsk_8drpB_AMNSB8Tnur3jrKkt9egkSfezH8xGuDcToYaCQ5e9qarYAmXqor2w6uY5ph9qV1s")

# Prepare data for insertion into Pinecone
vectors_to_insert = []
for i, embedding in enumerate(embeddings):
    vectors_to_insert.append((
        str(i),  # Unique ID for each vector
        embedding.tolist(),  # Embedding vector
        {"chunk_id": i, "content": chunks[i].page_content[:200]}  # Optional metadata
    ))

# Insert the embeddings into Pinecone
index.upsert(vectors=vectors_to_insert)

# Verify insertion by querying the Pinecone index (optional)
query_result = index.query(vector=embeddings[0].tolist(), top_k=3)  # Corrected to use keyword arguments
print("Query Result:", query_result)

# Optionally, check the status of the index
status = index.describe_index_stats()
print("Index Stats:", status)

Indexes: ['netsol-financials', 'itu']
Details of index 'netsol-financials': {'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'netsol-financials-c2umrz9.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'netsol-financials',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}
Query Result: {'matches': [{'id': '0', 'score': 1.00092947, 'values': []},
             {'id': 'f50c76eaf5514241d649ccc89fca3042',
              'score': 0.662193596,
              'values': []},
             {'id': '1', 'score': 0.640473843, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}
Index Stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2982}},
 'total_vector_count': 2982}


## Llama code

In [13]:
# Function to get a response from Groq (LLaMA model)
def get_response_from_lama(prompt: str, model: str = "llama3-8b-8192") -> str:
    """
    Fetch a response from the LLaMA model using the Groq client library.
    Assumes the Groq client is initialized and accessible as `client`.
    """
    try:
        # Call the Groq API using the client
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=0.7
        )
        # Return the content of the response
        return chat_completion.choices[0].message.content
    except Exception as e:
        # Raise an exception with details on failure
        raise ValueError(f"Groq API call failed: {e}")


In [15]:
# Initialize the Groq client
from groq import Groq

client = Groq(api_key="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")

# Call the function
prompt = "What is the capital of France?"
response = get_response_from_lama(prompt)
print(response)


The capital of France is Paris.


## Tavily code

In [ ]:
def get_tavily_response(query: str) -> str:
    """
    Fetch current event data using Tavily API via TavilyClient.
    Parses the response to extract relevant titles and URLs.
    """
    from tavily import TavilyClient

    # Initialize the Tavily client
    tavily_client = TavilyClient(api_key="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")

    try:
        # Perform the search query
        response = tavily_client.search(query)

        # Check for results and parse appropriately
        if "results" in response and len(response["results"]) > 0:
            # Extract and format the top results
            formatted_results = "\n\n".join(
                f"**Title**: {item['title']}\n**URL**: {item['url']}\n**Content Snippet**: {item['content'][:200]}..."
                for item in response["results"][:3]
            )
            return f"Top Results:\n{formatted_results}"
        else:
            return "No relevant results found for the query."
    except Exception as e:
        # Handle errors gracefully
        return f"Error fetching data from Tavily: {str(e)}"

# Example Usage
query = "What are the latest advancements in AI?"
response = get_tavily_response(query)
print(response)


Top Results:
**Title**: Who is Winning the AI Race in 2024? Big Tech's Race to AGI
**URL**: https://www.unite.ai/who-is-winning-the-ai-race-in-2024-big-techs-race-to-agi/
**Content Snippet**: Explore the latest advancements in AI by major tech giants like Microsoft, Meta, OpenAI, Anathropic, Amazon, and Apple. Understand the competitive landscape and the implications of AGI, quantum comput...

**Title**: AI for everything: 10 Breakthrough Technologies 2024
**URL**: https://www.technologyreview.com/2024/01/08/1085096/artificial-intelligence-generative-ai-chatgpt-open-ai-breakthrough-technologies
**Content Snippet**: 10 Breakthrough Technologies
by Will Douglas Heaven
Share
Popular
Deep Dive
Artificial intelligence
Google DeepMind used a large language model to solve an unsolved math problem
They had to throw away...

**Title**: The Top Artificial Intelligence Trends | IBM
**URL**: https://www.ibm.com/think/insights/artificial-intelligence-trends
**Content Snippet**: Here are some import

## Integrating Each Function in my RAG Workflow

In [19]:
import os
from groq import Groq
import numpy as np
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec, Index
from tavily import TavilyClient

# Set the API key for Groq
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_8drpB_AMNSB8Tnur3jrKkt9egkSfezH8xGuDcToYaCQ5e9qarYAmXqor2w6uY5ph9qV1s")  # Use your actual API key

# List indexes
indexes = pc.list_indexes().names()
# If an index exists, describe it, else create a new index
if indexes:
    index_name = indexes[0]  # Access the first index in the list and get the 'name'
    index_description = pc.describe_index(index_name)  # Pass the index name as a string
else:
    # Create a new index if none exist
    pc.create_index(
        name='notes',
        dimension=384,  # Update dimension according to your embedding model
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Set your region here
        )
    )

# Access the 'notes' index using the Index class
index = Index(name='netsol_financials', host='https://netsol-financials-c2umrz9.svc.aped-4627-b74a.pinecone.io', api_key="pcsk_8drpB_AMNSB8Tnur3jrKkt9egkSfezH8xGuDcToYaCQ5e9qarYAmXqor2w6uY5ph9qV1s")

# Load Sentence-BERT model for generating query embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to generate embeddings for user input
def generate_embeddings(texts):
    return model.encode(texts)

In [27]:

# Function to classify queries
def classify_query(query):
    if any(keyword in query.lower() for keyword in ["finance", "financial", "stock", "investment", "market", "NetSol"]):
        return "finance"
    elif any(keyword in query.lower() for keyword in ["news", "event", "current", "recent", "happening now", "live"]):
        return "live-event"
    else:
        return "general"

# # Function to generate embeddings for user input
# def generate_embeddings(texts):
#     return model.encode(texts)

def get_tavily_response(query: str) -> str:
    """
    Fetch current event data using Tavily API via TavilyClient.
    Parses the response to extract relevant titles and URLs.
    """
    from tavily import TavilyClient

    # Initialize the Tavily client
    tavily_client = TavilyClient(api_key="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")

    try:
        # Perform the search query
        response = tavily_client.search(query)

        # Check for results and parse appropriately
        if "results" in response and len(response["results"]) > 0:
            # Extract and format the top results
            formatted_results = "\n\n".join(
                f"**Title**: {item['title']}\n**URL**: {item['url']}\n**Content Snippet**: {item['content'][:200]}..."
                for item in response["results"][:3]
            )
            return f"Top Results:\n{formatted_results}"
        else:
            return "No relevant results found for the query."
    except Exception as e:
        # Handle errors gracefully
        return f"Error fetching data from Tavily: {str(e)}"

# Function to get a response from Groq (LLaMA model)
def get_response_from_lama(prompt: str, model: str = "llama3-8b-8192") -> str:
    """
    Fetch a response from the LLaMA model using the Groq client library.
    Assumes the Groq client is initialized and accessible as `client`.
    """
    try:
        # Call the Groq API using the client
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
            temperature=0.7
        )
        # Return the content of the response
        return chat_completion.choices[0].message.content
    except Exception as e:
        # Raise an exception with details on failure
        raise ValueError(f"Groq API call failed: {e}")

def get_answer_from_query(user_query: str):
    # Classify the query
    category = classify_query(user_query)
    query_embedding = generate_embeddings([user_query])[0]

    if category == "finance":
        # Query Pinecone with the embedding and get metadata if available
        results = index.query(vector=query_embedding.tolist(), top_k=3, include_metadata=True)

        # Extract the context from metadata if available
        if 'matches' in results:
            context = "\n".join([res['metadata']['content'] for res in results['matches'] if 'metadata' in res])
        else:
            context = "No context found in metadata."
    elif category == "live-event":
        context = get_tavily_response(user_query)
    else:
        return get_response_from_lama(f"User Query: {user_query}\nAnswer:")

    prompt = f"Context:\n{context}\nUser Query: {user_query}\nAnswer:"
    return get_response_from_lama(prompt)


# User input query
user_query = input("Please enter your query: ")

# Get the answer from the Pinecone + LLaMA pipeline
answer = get_answer_from_query(user_query)

# Print only the question and the answer
print(f"Question: {user_query}")
print(f"Answer: {answer}")
#NetSol

Please enter your query: Explain the key financial metrics for NetSol in 2024.
Question: Explain the key financial metrics for NetSol in 2024.
Answer: Based on the provided data, the key financial metrics for NetSol in 2024 are:

**Working Capital:**

* The working capital for NetSol in 2024 is:

1. 2024: ₹8,738,419
2. 2023: ₹7,623,439
3. 2022: ₹5,942,484
4. 2021: ₹4,762,877
5. 2020: ₹4,379,835
6. 2019: ₹3,739,850

Working capital represents the company's ability to pay its short-term debts and fund its daily operations. A higher working capital indicates a stronger financial position, while a lower working capital may indicate liquidity issues.

Please note that these figures are specific to the fiscal year 2024 and may not reflect the company's current financial situation.


## RAGAS METRICS

In [ ]:
print(dir(Groq))

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_build_headers', '_build_request', '_calculate_retry_timeout', '_default_stream_cls', '_enforce_trailing_slash', '_idempotency_key', '_is_protocol', '_make_sse_decoder', '_make_status_error', '_make_status_error_from_response', '_maybe_override_cast_to', '_parse_retry_after_header', '_prepare_options', '_prepare_request', '_prepare_url', '_process_response', '_process_response_data', '_request', '_request_api_list', '_retry_request', '_serialize_multipartform', '_should_retry', '_should_stream_response_body', '_validate_headers', 'aut

In [ ]:
import ragas
print(dir(ragas))


['EvaluationDataset', 'MultiTurnSample', 'RunConfig', 'SingleTurnSample', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_analytics', '_version', 'callbacks', 'cost', 'dataset_schema', 'embeddings', 'evaluate', 'evaluation', 'exceptions', 'executor', 'integrations', 'llms', 'losses', 'messages', 'metrics', 'prompt', 'run_config', 'sdk', 'utils', 'validation']


In [ ]:
from ragas import evaluate
print(dir(evaluate))


['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__wrapped__']


In [ ]:
import ragas.metrics
print(dir(ragas.metrics))


['AgentGoalAccuracyWithReference', 'AgentGoalAccuracyWithoutReference', 'AnswerCorrectness', 'AnswerRelevancy', 'AnswerSimilarity', 'AspectCritic', 'BleuScore', 'ContextEntityRecall', 'ContextPrecision', 'ContextRecall', 'ContextUtilization', 'DataCompyScore', 'DistanceMeasure', 'ExactMatch', 'FactualCorrectness', 'Faithfulness', 'FaithfulnesswithHHEM', 'InstanceRubrics', 'LLMContextPrecisionWithReference', 'LLMContextPrecisionWithoutReference', 'LLMContextRecall', 'LLMSQLEquivalence', 'Metric', 'MetricOutputType', 'MetricType', 'MetricWithEmbeddings', 'MetricWithLLM', 'MultiModalFaithfulness', 'MultiModalRelevance', 'MultiTurnMetric', 'NoiseSensitivity', 'NonLLMContextPrecisionWithReference', 'NonLLMContextRecall', 'NonLLMStringSimilarity', 'ResponseRelevancy', 'RougeScore', 'RubricsScore', 'SemanticSimilarity', 'SimpleCriteriaScore', 'SingleTurnMetric', 'StringPresence', 'SummarizationScore', 'ToolCallAccuracy', 'TopicAdherenceScore', '__all__', '__builtins__', '__cached__', '__doc__

In [ ]:
from ragas.metrics import AnswerCorrectness

correctness_metric = AnswerCorrectness()
print(dir(correctness_metric))


['__abstractmethods__', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_ascore', '_compute_statement_presence', '_create_simplified_statements', '_get_prompts', '_only_required_columns_single_turn', '_optimize_demonstration', '_optimize_instruction', '_required_columns', '_single_turn_ascore', 'adapt_prompts', 'answer_similarity', 'ascore', 'beta', 'correctness_prompt', 'embeddings', 'get_prompts', 'get_required_columns', 'init', 'llm', 'load_prompts', 'long_form_answer_prompt', 'max_retries', 'name', 'output_type', 'required_columns', 'save_prompts', 'score', 'senten

In [ ]:
help(correctness_metric.single_turn_score)


Help on method single_turn_score in module ragas.metrics.base:

single_turn_score(sample: 'SingleTurnSample', callbacks: 'Callbacks' = None) -> 'float' method of ragas.metrics._answer_correctness.AnswerCorrectness instance
    Synchronously score a single-turn sample.
    
    May raise ImportError if nest_asyncio is not installed in a Jupyter-like environment.



In [ ]:
!pip install --upgrade ragas


## Ealuating Answercorrectness and Faithfulness of Random Example

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def evaluate_qa(data):
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Load a pre-trained sentence transformer
    total_questions = len(data)
    correct = 0
    faithfulness = 0

    for entry in data:
        question = entry.get('question', '')
        answer = entry.get('answer', '')
        ground_truth = entry.get('ground_truth', '')
        generated_answer = entry.get('generated_answer', '')

        # Correctness: Compare answer with ground truth
        if answer.strip().lower() == ground_truth.strip().lower():
            correct += 1

        # Faithfulness: Compare generated answer with ground truth using semantic similarity
        if generated_answer and ground_truth:
            embeddings = model.encode([generated_answer, ground_truth])
            similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
            faithfulness += similarity

    # Calculate metrics
    correctness_score = correct / total_questions * 100
    faithfulness_score = faithfulness / total_questions * 100

    return {
        "total_questions": total_questions,
        "correctness_score": correctness_score,
        "faithfulness_score": faithfulness_score
    }

# Example
data = [
    {
        "question": "What is the vision of NETSOL Technologies Limited?",
        "answer": "To become the leading and world-class provider of IT solutions...",
        "generated_answer": "The goal is to be a world-class provider of IT solutions...",
        "ground_truth": "To become the leading and world-class provider of IT solutions..."
    }
]
results = evaluate_qa(data)
print("Evaluation Results:", results)


Evaluation Results: {'total_questions': 1, 'correctness_score': 100.0, 'faithfulness_score': 93.33614110946655}


## Ealuating (RAGAS Metrics) Answercorrectness and Faithfulness of (Question Answer pairs) qa_data.json

I use sklearn's cosine similarity because ragas.metrics giving me unresolvable errors. I use the same formula to calculate the answer correctness and faithfullness as ragas.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json

def evaluate_qa(data):
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Load a pre-trained sentence transformer
    total_questions = len(data)
    correct = 0
    faithfulness = 0

    for entry in data:
        question = entry.get('question', '')
        retrieved_context = entry.get('retrieved_context', '')
        generated_answer = entry.get('generated_answer', '')
        ground_truth = entry.get('ground_truth', '')

        # Correctness: Compare answer with ground truth
        if retrieved_context.strip().lower() == ground_truth.strip().lower():
            correct += 1

        # Faithfulness: Compare generated answer with ground truth using semantic similarity
        if generated_answer and ground_truth:
            embeddings = model.encode([generated_answer, ground_truth])
            similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
            faithfulness += similarity

    # Calculate metrics
    correctness_score = correct / total_questions * 100
    faithfulness_score = faithfulness / total_questions * 100

    return {
        "Total_questions": total_questions,
        "Answercorrectness": correctness_score,
        "Faithfulness": faithfulness_score
    }

# Example usage
with open("qa_data_copy.json", 'r') as f:
    qa_data = json.load(f)

results = evaluate_qa(qa_data)
print("Evaluation Results:", results)


Evaluation Results: {'Total_questions': 22, 'Answercorrectness': 100.0, 'Faithfulness': 58.58516081828963}


## Gradio App

In [1]:
!pip install gradio


In [28]:
import gradio as gr
# Function to create a Gradio app
def create_gradio_app():
    with gr.Blocks() as app:
        gr.Markdown("## Retrieval-Augmented Generation Chatbot")
        query = gr.Textbox(label="Enter your query")
        output = gr.Textbox(label="Answer")

        def on_query(query):
            return get_answer_from_query(query)

        query.submit(on_query, inputs=query, outputs=output)

    return app

In [35]:
# Create Gradio app and launch it
app = create_gradio_app()
app.launch(server_name="0.0.0.0", server_port=7865)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8face6260dbbb564f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8face6260dbbb564f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

#Gradio App is deployed on huggingface spaces and the link is shared in my submission and also in my git repository.
